In [6]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import zipfile
import os

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /content/validation-horse-or-human.zip

--2020-07-29 14:29:35--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.195.128, 74.125.124.128, 172.217.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘/content/validation-horse-or-human.zip’

/content/validation 100%[===================>]  10.95M  --.-KB/s    in 0.1s    

2020-07-29 14:29:35 (102 MB/s) - ‘/content/validation-horse-or-human.zip’ saved [11480187/11480187]



In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /content/horse-or-human.zip

--2020-07-29 14:29:36--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.114.128, 74.125.124.128, 172.217.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.114.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/content/horse-or-human.zip’

/content/horse-or-h 100%[===================>] 142.65M   143MB/s    in 1.0s    

2020-07-29 14:29:37 (143 MB/s) - ‘/content/horse-or-human.zip’ saved [149574867/149574867]



In [5]:
train_zip = '/content/horse-or-human.zip'
valid_zip = '/content/validation-horse-or-human.zip'

train_dir = '/content/horse-or-human'
valid_dir = '/content/validation-horse-or-human'

with zipfile.ZipFile(train_zip, 'r') as zip_ref:
    zip_ref.extractall(train_dir)

with zipfile.ZipFile(valid_zip, 'r') as zip_ref:
    zip_ref.extractall(valid_dir)

In [8]:
train_human = os.path.join(train_dir, 'humans')
train_horse = os.path.join(train_dir, 'horses')

valid_human = os.path.join(valid_dir, 'humans')
valid_horse = os.path.join(valid_dir, 'horses')

In [11]:
print(f'There are {len(os.listdir(train_human))} human training images')
print(f'There are {len(os.listdir(train_horse))} horse training images')

print(f'There are {len(os.listdir(valid_human))} human validation images')
print(f'There are {len(os.listdir(valid_horse))} horse validation images')

There are 527 human training images
There are 500 horse training images
There are 128 human validation images
There are 128 horse validation images


In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [16]:
train_datagen = ImageDataGenerator(
    rescale = 1/255.,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip= True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(
    rescale=1/255.
)

In [32]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(300,300), batch_size=128, class_mode='binary')

valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(300,300), batch_size=32, class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [41]:
#Define the model
model = tf.keras.models.Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [45]:

model.compile(loss='binary_crossentropy', optimizer = tf.keras.optimizers.RMSprop(0.001), metrics=['accuracy'])

In [46]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('accuracy') > 0.99):
      self.model.stop_training = True

In [ ]:
callback = myCallback()

history = model.fit(
    train_generator,
    validation_data = valid_generator,
    epochs = 20,
    callbacks = [callback], 
    verbose = 1
)

Epoch 1/20
9/9 [==============================] - 29s 3s/step - loss: 2.0023 - accuracy: 0.7303 - val_loss: 1.6271 - val_accuracy: 0.5195
Epoch 2/20
9/9 [==============================] - 26s 3s/step - loss: 0.4147 - accuracy: 0.8199 - val_loss: 2.0313 - val_accuracy: 0.5977
Epoch 3/20
9/9 [==============================] - 26s 3s/step - loss: 0.4433 - accuracy: 0.7858 - val_loss: 1.5617 - val_accuracy: 0.5547
Epoch 4/20
9/9 [==============================] - 29s 3s/step - loss: 0.3412 - accuracy: 0.8452 - val_loss: 3.5557 - val_accuracy: 0.5039
Epoch 5/20
9/9 [==============================] - 29s 3s/step - loss: 0.4560 - accuracy: 0.7858 - val_loss: 1.9461 - val_accuracy: 0.5234
Epoch 6/20
9/9 [==============================] - 26s 3s/step - loss: 0.3639 - accuracy: 0.8462 - val_loss: 2.3536 - val_accuracy: 0.5547
Epoch 7/20
9/9 [==============================] - 25s 3s/step - loss: 0.2830 - accuracy: 0.8647 - val_loss: 4.2110 - val_accuracy: 0.5273
Epoch 8/20
9/9 [==================